<a href="https://colab.research.google.com/github/biniwollo/Fall24-DSA-5900/blob/main/Federated_TS_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  #import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.14.1


In [ ]:
pip install tensorflow_federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [24]:
import yfinance as yf
import os
import pandas as pd
import matplotlib.pyplot as plt
import keras
import numpy as np
import collections
import tensorflow_federated as tff

In [ ]:
# Define the companies and their ticker symbols
companies = {
    'John Deere': 'DE',
    'Archer-Daniels-Midland': 'ADM',
    'Bunge Ltd': 'BG',
    'The Mosaic Company': 'MOS',
    'Corteva': 'CTVA'
}

# Set up directory in the default Colab environment
base_dir = '/content/FinancialData'
os.makedirs(base_dir, exist_ok=True)

stock_data_dict = {}

# Loop through each company and download the stock data
for company, ticker in companies.items():
    print(f"Downloading data for {company} ({ticker})...")
    stock_data = yf.download(ticker, start='2019-09-16', end='2024-09-13')
    file_path = os.path.join(base_dir, f"{ticker}_stock_data.csv")
    stock_data.to_csv(file_path)
    stock_data.columns = stock_data.columns.droplevel(1)
    stock_data_dict[ticker] = stock_data
    print(f"Data for {company} ({ticker}) saved successfully at {file_path}")

# Combine all data into a single CSV (optional)
combined_file_path = os.path.join(base_dir, "combined_stock_data.csv")
combined_stock_data = pd.concat([pd.read_csv(os.path.join(base_dir, f"{ticker}_stock_data.csv")) for ticker in companies.values()])
combined_stock_data.to_csv(combined_file_path, index=False)
print(f"Combined stock data saved at: {combined_file_path}")

[*********************100%***********************]  1 of 1 completed


Data for John Deere (DE) saved successfully at /content/FinancialData/DE_stock_data.csv


[*********************100%***********************]  1 of 1 completed


Data for Archer-Daniels-Midland (ADM) saved successfully at /content/FinancialData/ADM_stock_data.csv


[*********************100%***********************]  1 of 1 completed


Data for Bunge Ltd (BG) saved successfully at /content/FinancialData/BG_stock_data.csv


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Data for The Mosaic Company (MOS) saved successfully at /content/FinancialData/MOS_stock_data.csv
Data for Corteva (CTVA) saved successfully at /content/FinancialData/CTVA_stock_data.csv
Combined stock data saved at: /content/FinancialData/combined_stock_data.csv


In [ ]:
def splitDF(df):
  column_indices = {name: i for i, name in enumerate(df.columns)}

  n = len(df)
  train_df = df[0:int(n*0.7)]
  val_df = df[int(n*0.7):int(n*0.9)]
  test_df = df[int(n*0.9):]

  num_features = df.shape[1]

  train_mean = train_df.mean()
  train_std = train_df.std()

  train_df = (train_df - train_mean) / train_std
  val_df = (val_df - train_mean) / train_std
  test_df = (test_df - train_mean) / train_std

  return train_df, val_df, test_df

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
def plot(self, model=None, plot_col='Close', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normalized]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Day')
  plt.show()

WindowGenerator.plot = plot

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    return tf.tile(inputs[:, -1:, :], [1, 50, 1])

def Baseline1(window, multi_val_performance, multi_performance):
  last_baseline = MultiStepLastBaseline()
  last_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.MeanAbsoluteError()])

  multi_val_performance['Baseline'] = last_baseline.evaluate(window.val, return_dict=True)
  multi_performance['Baseline'] = last_baseline.evaluate(window.test, verbose=0, return_dict=True)
  window.plot(last_baseline)

In [ ]:
class RepeatBaseline(tf.keras.Model):
  def call(self, inputs):
    return inputs

def Baseline2(window, multi_val_performance, multi_performance):
  repeat_baseline = RepeatBaseline()
  repeat_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                          metrics=[tf.keras.metrics.MeanAbsoluteError()])

  multi_val_performance['Repeat'] = repeat_baseline.evaluate(window.val, return_dict=True)
  multi_performance['Repeat'] = repeat_baseline.evaluate(window.test, verbose=0, return_dict=True)
  window.plot(repeat_baseline)


In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
def LinearModel(window, outSteps, multi_val_performance, multi_performance):
  num_features = 1
  linear_model = tf.keras.Sequential([
      # Take the last time-step.
      # Shape [batch, time, features] => [batch, 1, features]
      tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
      # Shape => [batch, 1, out_steps*features]
      tf.keras.layers.Dense(outSteps*num_features,
                            kernel_initializer=tf.initializers.zeros()),
      # Shape => [batch, out_steps, features]
      tf.keras.layers.Reshape([outSteps, num_features])
  ])

  history = compile_and_fit(linear_model, window)

  multi_val_performance['Linear'] = linear_model.evaluate(window.val, return_dict=True)
  multi_performance['Linear'] = linear_model.evaluate(window.test, verbose=0, return_dict=True)
  window.plot(linear_model)

In [ ]:
def DenseModel(window, outSteps, multi_val_performance, multi_performance):
  num_features = 1
  multi_dense_model = tf.keras.Sequential([
      # Take the last time step.
      # Shape [batch, time, features] => [batch, 1, features]
      tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
      # Shape => [batch, 1, dense_units]
      tf.keras.layers.Dense(512, activation='relu'),
      # Shape => [batch, out_steps*features]
      tf.keras.layers.Dense(outSteps*num_features,
                          kernel_initializer=tf.initializers.zeros()),
      # Shape => [batch, out_steps, features]
      tf.keras.layers.Reshape([outSteps, num_features])])

  history = compile_and_fit(multi_dense_model, window)

  multi_val_performance['Dense'] = multi_dense_model.evaluate(window.val, return_dict=True)
  multi_performance['Dense'] = multi_dense_model.evaluate(window.test, verbose=0, return_dict=True)
  window.plot(multi_dense_model)

In [ ]:
def ConvModel(window, outSteps, convWidth, multi_val_performance, multi_performance):
  num_features=1
  multi_conv_model = tf.keras.Sequential([
      # Shape [batch, time, features] => [batch, convWidth, features]
      tf.keras.layers.Lambda(lambda x: x[:, -convWidth:, :]),
      # Shape => [batch, 1, conv_units]
      tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(convWidth)),
      # Shape => [batch, 1,  out_steps*features]
      tf.keras.layers.Dense(outSteps*num_features,
                            kernel_initializer=tf.initializers.zeros()),
      # Shape => [batch, out_steps, features]
      tf.keras.layers.Reshape([outSteps, num_features])
  ])

  history = compile_and_fit(multi_conv_model, window)

  multi_val_performance['Conv'] = multi_conv_model.evaluate(window.val, return_dict=True)
  multi_performance['Conv'] = multi_conv_model.evaluate(window.test, verbose=0, return_dict=True)
  window.plot(multi_conv_model)

In [ ]:
def LSTM(window, outSteps, multi_val_performance, multi_performance):
  num_features=1
  multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(outSteps*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([outSteps, num_features])
  ])

  history = compile_and_fit(multi_lstm_model, window)

  multi_val_performance['LSTM'] = multi_lstm_model.evaluate(window.val, return_dict=True)
  multi_performance['LSTM'] = multi_lstm_model.evaluate(window.test, verbose=0, return_dict=True)
  window.plot(multi_lstm_model)

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(6)

def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the LSTM state.
  prediction, state = self.warmup(inputs)

  # Insert the first prediction.
  predictions.append(prediction)

  # Run the rest of the prediction steps.
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output.
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call

def AR_LSTM(window, outSteps, multi_val_performance, multi_performance):
  feedback_model = FeedBack(units=32, out_steps=outSteps)
  history = compile_and_fit(feedback_model, window)

  multi_val_performance['AR LSTM'] = feedback_model.evaluate(window.val, return_dict=True)
  multi_performance['AR LSTM'] = feedback_model.evaluate(window.test, verbose=0, return_dict=True)
  window.plot(feedback_model)

In [ ]:
def printModelSummary(multi_val_performance, multi_performance):
  x = np.arange(len(multi_performance))
  width = 0.3

  metric_name = 'mean_absolute_error'
  val_mae = [v[metric_name] for v in multi_val_performance.values()]
  test_mae = [v[metric_name] for v in multi_performance.values()]

  plt.bar(x - 0.17, val_mae, width, label='Validation')
  plt.bar(x + 0.17, test_mae, width, label='Test')
  plt.xticks(ticks=x, labels=multi_performance.keys(),
            rotation=45)
  plt.ylabel(f'MAE (average over all times and outputs)')
  _ = plt.legend()
  plt.show()

In [113]:

OUT_STEPS = 50
CONV_WIDTH = 3
window = WindowGenerator(input_width=50, label_width=OUT_STEPS, shift=OUT_STEPS, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=['Close'])
window

client_train_dataset = dict()
client_test_dataset = dict()
client_ids = []
for ticker, df in stock_data_dict.items():
  train_df, val_df, test_df = splitDF(df)
  OUT_STEPS = 50
  CONV_WIDTH = 3
  window = WindowGenerator(input_width=50, label_width=OUT_STEPS, shift=OUT_STEPS, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=['Close'])
  client_train_dataset[ticker] = window.train
  client_test_dataset[ticker] = window.test
  client_ids.append(ticker)

client_train_dataset


{'DE': <_MapDataset element_spec=(TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 50, 1), dtype=tf.float32, name=None))>,
 'ADM': <_MapDataset element_spec=(TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 50, 1), dtype=tf.float32, name=None))>,
 'BG': <_MapDataset element_spec=(TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 50, 1), dtype=tf.float32, name=None))>,
 'MOS': <_MapDataset element_spec=(TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 50, 1), dtype=tf.float32, name=None))>,
 'CTVA': <_MapDataset element_spec=(TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 50, 1), dtype=tf.float32, name=None))>}

In [78]:
def create_dataset(client):
  return client_train_dataset[client]

client_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(client_ids, create_dataset)
client_data

In [79]:
len(client_data.client_ids)

5

In [89]:
example_dataset = client_data.create_tf_dataset_for_client(
    client_data.client_ids[0])

example_element = next(iter(example_dataset))
example_element

(<tf.Tensor: shape=(32, 50, 6), dtype=float32, numpy=
 array([[[-0.6470073 , -0.64284134, -0.6628972 , -0.6248717 ,
          -0.63524795, -0.60462964],
         [-0.64004076, -0.63561296, -0.6632033 , -0.6309277 ,
          -0.6455614 , -0.5116219 ],
         [-0.62760085, -0.62270504, -0.6317782 , -0.6037803 ,
          -0.63266957, -0.81410986],
         ...,
         [-0.3682542 , -0.3536008 , -0.38711154, -0.34848994,
          -0.36823204, -0.25180805],
         [-0.3068507 , -0.2898872 , -0.31762955, -0.31695727,
          -0.3479145 ,  1.8747263 ],
         [-0.3362098 , -0.32035   , -0.31201804, -0.30630723,
          -0.33842596,  0.50441194]],
 
        [[-1.272003  , -1.2657259 , -1.2811955 , -1.2465479 ,
          -1.2688047 , -0.09570089],
         [-1.3077176 , -1.3033137 , -1.2973162 , -1.2793337 ,
          -1.2750959 , -0.09363405],
         [-1.3214538 , -1.3177705 , -1.3153753 , -1.2978148 ,
          -1.2960323 , -0.54274607],
         ...,
         [-1.636117  , -

In [90]:
def make_federated_data(client_data, client_ids):
  return [
      client_data.create_tf_dataset_for_client(x)
      for x in client_ids
  ]

In [92]:
federated_train_data = make_federated_data(client_data, client_ids)

print(f'Number of client datasets: {len(federated_train_data)}')
print(f'First dataset: {federated_train_data[0]}')

Number of client datasets: 5
First dataset: <_MapDataset element_spec=(TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 50, 1), dtype=tf.float32, name=None))>


In [93]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  num_features = 1
  linear_model = tf.keras.Sequential([
      # Take the last time-step.
      tf.keras.layers.InputLayer(input_shape=(50,6)),
      # Shape [batch, time, features] => [batch, 1, features]
      tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
      # Shape => [batch, 1, out_steps*features]
      tf.keras.layers.Dense(OUT_STEPS*num_features,
                            kernel_initializer=tf.initializers.zeros()),
      # Shape => [batch, out_steps, features]
      tf.keras.layers.Reshape([OUT_STEPS, num_features])
  ])

  return tff.learning.models.from_keras_model(
      linear_model,
      input_spec=client_train_dataset['DE'].element_spec,
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [99]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=0.02),
    server_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=1.0))

In [100]:
print(training_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[6,50],
      float32[50]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<
    learning_rate=float32
  >,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    learning_rate=float32
  >
>@SERVER)


In [107]:
train_state = training_process.initialize()

In [108]:
result = training_process.next(train_state, federated_train_data)
train_state = result.state
train_metrics = result.metrics
print('round  1, metrics={}'.format(train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.8007774), ('loss', 0.89618546), ('num_examples', 3900), ('num_batches', 125)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [109]:
NUM_ROUNDS = 25
for round_num in range(2, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.73515666), ('loss', 0.7599916), ('num_examples', 3900), ('num_batches', 125)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.6759308), ('loss', 0.647527), ('num_examples', 3900), ('num_batches', 125)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.6224728), ('loss', 0.5546526), ('num_examples', 3900), ('num_batches', 125)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('up

In [110]:
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)
print(evaluation_process.next.type_signature.formatted_representation())

(<
  state=<
    global_model_weights=<
      trainable=<
        float32[6,50],
        float32[50]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        mean_absolute_error=<
          float32,
          float32
        >,
        loss=<
          float32,
          float32
        >,
        num_examples=<
          int64
        >,
        num_batches=<
          int64
        >
      >
    >,
    aggregator=<
      value_sum_process=<>,
      weight_sum_process=<>
    >,
    finalizer=<>
  >@SERVER,
  client_data={<
    float32[?,50,6],
    float32[?,50,1]
  >*}@CLIENTS
> -> <
  state=<
    global_model_weights=<
      trainable=<
        float32[6,50],
        float32[50]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        mean_absolute_error=<
          float32,
          float32
        >,
        loss=<
          float32,
          float32
        >,
        num_examples=<
     

In [111]:
evaluation_state = evaluation_process.initialize()
model_weights = training_process.get_model_weights(train_state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

In [114]:
def create_test_dataset(client):
  return client_test_dataset[client]

client_test_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(client_ids, create_test_dataset)

federated_test_data = make_federated_data(client_test_data, client_ids)

len(federated_test_data), federated_test_data[0]

(5,
 <_MapDataset element_spec=(TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 50, 1), dtype=tf.float32, name=None))>)

In [115]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

In [116]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('mean_absolute_error', 0.15457316), ('loss', 0.036595188), ('num_examples', 135), ('num_batches', 5)])), ('total_rounds_metrics', OrderedDict([('mean_absolute_error', 0.15457316), ('loss', 0.036595188), ('num_examples', 135), ('num_batches', 5)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"